In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!rm spark-3.5.1-bin-hadoop3.tgz   # Tidying up

--2025-03-25 00:39:54--  https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400446614 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.1-bin-hadoop3.tgz’

spark-3.5.1-bin-had 100%[===================>] 381.90M  23.0MB/s    in 20s     

2025-03-25 00:40:15 (19.1 MB/s) - ‘spark-3.5.1-bin-hadoop3.tgz’ saved [400446614/400446614]



In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [4]:
!pip install -q findspark
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [7]:
from google.colab import files

uploaded = files.upload()

Saving crimeTable.csv to crimeTable (1).csv


In [ ]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Read the uploaded CSV file
df = spark.read.csv('/content/crimeTable.csv', header=True, inferSchema=True)

# Check the schema to ensure the column names are correct
df.printSchema()


root
 |-- DR_NO: integer (nullable = true)
 |-- Date_Rptd: string (nullable = true)
 |-- DATE_OCC: string (nullable = true)
 |-- TIME_OCC: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- AREA_NAME: string (nullable = true)
 |-- Rpt_Dist_No: integer (nullable = true)
 |-- Part_1_2: integer (nullable = true)
 |-- Crm_Cd: integer (nullable = true)
 |-- Crm_Cd_Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict_Age: integer (nullable = true)
 |-- Vict_Sex: string (nullable = true)
 |-- Vict_Descent: string (nullable = true)
 |-- Premis_Cd: integer (nullable = true)
 |-- Premis_Desc: string (nullable = true)
 |-- Weapon_Used_Cd: integer (nullable = true)
 |-- Weapon_Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status_Desc: string (nullable = true)
 |-- Crm_Cd_1: integer (nullable = true)
 |-- Crm_Cd_2: integer (nullable = true)
 |-- Crm_Cd_3: integer (nullable = true)
 |-- Crm_Cd_4: integer (nullable = true)
 |-- L

In [8]:
# Groups crime description by victim demographic

rdd = df.select("Crm_Cd_Desc","Vict_Sex","Vict_Descent","Vict_Age").rdd

def renameCrimes(crime):
    crimeDesc = crime["Crm_Cd_Desc"] if crime["Crm_Cd_Desc"] else "NA"
    age = crime["Vict_Age"]
    Crime_Category = "Other Crimes"
    ageDesc = "NA"

    if any(keyword in crimeDesc for keyword in ["ASSAULT","BATTERY","HOMICIDE"]):
        Crime_Category= "Violent Crimes"
    elif any(keyword in crimeDesc for keyword in ["RAPE","SEXUAL","LEWD"]):
        Crime_Category= "Sex Crimes"
    elif any(keyword in crimeDesc for keyword in ["BURGLARY","THEFT","SHOPLIFT"]):
        Crime_Category= "Property Crimes"
    elif any(keyword in crimeDesc for keyword in ["STOLEN","VEHICLE","AUTO"]):
        Crime_Category= "Vehicle-Related Crimes"
    elif any(keyword in crimeDesc for keyword in ["FIREARM","WEAPON"]):
        Crime_Category= "Weapons & Explosives"
    elif any(keyword in crimeDesc for keyword in ["CHILD","MINOR"]):
        Crime_Category= "Crimes Against Children"
    elif any(keyword in crimeDesc for keyword in ["FRAUD","FORGERY"]):
        Crime_Category= "Fraud & Financial Crimes"
    elif any(keyword in crimeDesc for keyword in ["DRUG","ALCOHOL"]):
        Crime_Category= "Drug & Alcohol-Related Crimes"
    elif any(keyword in crimeDesc for keyword in ["TRESPASS","DISTURBING"]):
        Crime_Category= "Public Disturbance"
    elif any(keyword in crimeDesc for keyword in ["TRAFFICKING","EXPLOITATION"]):
        Crime_Category= "Human Trafficking & Exploitation"
    elif any(keyword in crimeDesc for keyword in ["THREAT","HARASS"]):
        Crime_Category = "Threats & Harassment"
    elif any(keyword in crimeDesc for keyword in ["COURT", "VIOLATION"]):
        Crime_Category= "Court & Legal Violations"


    if 1 <= age <= 12:
      ageDesc = "Child (1-12)"
    elif 13 <= age <= 17:
      ageDesc = "Teen (13-17)"
    elif 18 <= age <= 25:
      ageDesc = "Young Adult (18-25)"
    elif 26 <= age <= 40:
      ageDesc = "Adult (26-40)"
    elif 41 <= age <= 60:
      ageDesc = "Middle-Aged"
    elif age>= 60:
      ageDesc = "Senior (60+)"
    else:
      ageDesc = "NA"

    return ((Crime_Category, crime["Vict_Sex"], crime["Vict_Descent"], ageDesc), 1)

rddMapped = rdd.map(renameCrimes)
rddReduced = rddMapped.reduceByKey(lambda a,b: a+b)
print(rddReduced.collect())






NameError: name 'df' is not defined